# Data preparation: Tampered images   

### LuNoTim-DeepInpainting: Removed Nodules

In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage import measure

pathRemoved = 'removed_ct_gan_inpainting/LIDC-IDRI'  

sliceData = [] 

dfRemoved = pd.DataFrame()
    
for root, dirs, files in os.walk(pathRemoved):
    listFiles = []
    dicID = {}
    for file in files:
        if 'mask' in file:
            
            pathMask = os.path.join(root, file)
            #print(root)
            
            #-------------Change the fileName to our desired type--------
            sliceName = str(int(file.split('_')[0])) + '.png'
            rootName = "%04d" %(int((root.split('-')[-1])))
            file = 'tp_CTGAN_rem_' + "LIDC-IDRI-" + rootName + '_' + sliceName
            #print(file)
            
            #-------------------------Load Mask---------------------------
            sliceMask = (np.load(pathMask)).astype(float)
           
            #----------------------Extract Features-----------------------
            labels = measure.label(sliceMask)
            regions = measure.regionprops(labels)
            
            for i in range(len(regions)):
                bbox = regions[i].bbox     # (min_row, min_col, max_row, max_col)
            
            
            #-----Add the slice mask information to the sliceData list----------
            # Class ------> prstine = '0', Tampered = '1'
            
            sliceData.append([file, bbox[2]- bbox[0], bbox[3]- bbox[1], 1, 
                              bbox[0], bbox[1], bbox[2], bbox[3]])
            #print(sliceData)
        
    tamperedDF = pd.DataFrame(sliceData, columns = ['filename', 'width', 'height', 'class', 
                                                    'xmin', 'ymin', 'xmax', 'ymax'])
    
    
    #tamperedDF.to_csv('removed_CTGAN', index=False)
    dfRemoved = tamperedDF.copy()

print('Done..............')     

### LuNoTim-DeepInpainting: Added Nodules

In [ ]:
pathAdded = 'added_ct_gan_inpainting/LIDC-IDRI/'

sliceData = [] 

dfAdded = pd.DataFrame()

for root, subdirectories, filenames in os.walk(pathAdded):
    for subdirectory in subdirectories:
        #dictionary for keeping patient ID and tampered Slices
        dicID = {}         
        listFiles = []    #keep List of tampered slices
        
        if root not in dicID.keys():
            temp = root
        
    for file in filenames:
        #Select mask files 
        if 'mask' in file:
            pathMask = os.path.join(root, file)
            #print(file)
            
            #---------------Change the fileName to our desired type
            sliceName = str(int(file.split('_')[0])) + '.png'
            #print(sliceName)
            rootName = (root.split('LIDC-IDRI/')[-1]).split('/')[0]
            file = 'tp-CTGAN-add-' + "LIDC-IDRI-" + rootName + '-' + sliceName
            #print(file)

            
            #-------------------------Load Mask---------------------------
            sliceMask = (np.load(pathMask)).astype(float)
            
            #-------------Apply Morphological Operators-------------------
            kernel = np.ones((2,2), 'uint8')
            dilate = cv2.morphologyEx(sliceMask, cv2.MORPH_DILATE, kernel)
            
            #----------------------Extract Features-----------------------
            labels = measure.label(dilate)
            regions = measure.regionprops(labels)
            
            for i in range(len(regions)):
                bbox = regions[i].bbox     # (min_row, min_col, max_row, max_col)
                
                
            #Save the features into a list
            sliceData.append([file, bbox[2]- bbox[0], bbox[3]- bbox[1], 1, 
                              bbox[0], bbox[1], bbox[2], bbox[3]])
            #print(sliceData)
                
    tamperedDF = pd.DataFrame(sliceData, columns = ['filename', 'width', 'height', 'class', 
                                                    'xmin', 'ymin', 'xmax', 'ymax'])
    
    #tamperedDF.to_csv('added_CTGAN', index=False)
    dfAdded = tamperedDF.copy()

### Merge data

In [ ]:
# Merge data
df = dfAdded.append(dfRemoved, ignore_index=True)
df.to_csv('tampered_CTGAN.csv', index=False)
